In [2]:
import pandas as pd



df = pd.read_csv('lora.tsv', sep='\t')

df.head()

/var/folders/6j/7myxq8l12wz8s_qhqdqq6d4r0000gn/T/ipykernel_56553/3308735501.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('lora.tsv', sep='\\t')


,Question\tAnswer
0,What is the key challenge with full fine-tunin...
1,What is Low-Rank Adaptation (LoRA)?\tLoRA is a...
2,How does LoRA compare to full fine-tuning in t...
3,What is the impact of LoRA on inference latenc...
4,Can LoRA be combined with other adaptation met...


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings.gpt4all import GPT4AllEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import NLTKTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
    # separators='.'
)

# loader = TextLoader("mlops.txt")
loader = PyPDFLoader("lora.pdf")
# pages = loader.load_and_split()
documents = loader.load()
# nltk_splitter = NLTKTextSplitter()
texts = text_splitter.split_documents(documents)
# texts = nltk_splitter.split_documents(documents)

In [3]:
from langchain.vectorstores import FAISS

embedder = GPT4AllEmbeddings()
# db = FAISS.from_documents(texts, embedding=embedder)
db = FAISS.load_local("ml_papers_2", embedder)
# db = FAISS.from_embeddings
# db.save_local('db')

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [3]:
query = "What is lora"
docs = db.similarity_search(query, k=5)

In [4]:
print(docs[0].page_content)

/s per V100 GPU for LoRA.
Model & Method


In [4]:
def retrieve_info(query):
    similar_response = db.similarity_search(query, k=3)

    page_contents_array = [doc.page_content for doc in similar_response]

    # print(page_contents_array)

    return page_contents_array


In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

local_path = (
    # "/Users/random/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin"  # replace with your desired local file path
    "./gpt4all.gguf"  # replace with your desired local file path
)

callbacks = [StreamingStdOutCallbackHandler()]

llm = GPT4All(
    model=local_path,
    backend="llama",
    verbose=True,
    streaming=True,
    callbacks=callbacks,
    max_tokens=1024,
)

# template = """You are a helpful AI assistant and provide the answer for the question based on the given context and your existing knowledge.
# Context:{context}
# >>QUESTION<<{message}
# >>ANSWER<<"""

template = """
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Your answers are always brief. Base your answers on your knowledge and the context.
<</SYS>>
Context:{context}
{message}[/INST]
"""

prompt = PromptTemplate(input_variables=["context", "message"], template=template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
def generate_response(message):
    context = retrieve_info(message)
    # print(context)
    response = llm_chain.run(message=message, context=context)
    return response

In [8]:
message = "what is qlora"
context = retrieve_info(message)
print(context)
response = llm_chain.run(message=message, context=context)

['//github.com/artidoro/qlora and https://github.com/TimDettmers', 'netuning. Furthermore, we want to analyze the components of QLoRA including the impact of\nNormalFloat4 over standard Float4. The following sections will discuss the experiments that aimed\nat answering these questions.\nhttps:', '/65B\nQLORA tuning on a single 24']

QLoRA (Quantized Linear Regression Autoencoder) is a deep learning model designed for efficient and accurate text generation. It uses a combination of linear regression and autoencoders to generate high-quality text while minimizing the computational resources required. QLoRA can be used in various natural language processing tasks, such as text summarization or text generation.